In [1]:
import os
import pandas as pd
import dask.dataframe as dd

In [ ]:
def load_data(directory: str) -> pd.DataFrame:

    all_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                all_files.append(os.path.join(root, file))

    if not all_files:
        raise FileNotFoundError("No CSV files found in the given directory.")

    df = dd.read_csv(all_files)
    return df.compute()

data_path = "./Data/individual_book_train"
df = load_data(data_path)
df = df[1000:2000]

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.0
mean,44.952000,297.335000,0.999772,1.000027,0.999532,1.000267,651.265000,653.451000,808.610000,953.112000,13.0
std,19.363591,186.650688,0.001294,0.001308,0.001296,0.001306,379.758109,437.577636,362.965739,513.037563,0.0
min,16.000000,0.000000,0.996742,0.997003,0.996481,0.997263,1.000000,1.000000,201.000000,200.000000,13.0
25%,31.000000,119.500000,0.999173,0.999409,0.998937,0.999646,400.000000,386.000000,600.000000,565.000000,13.0
50%,31.000000,290.500000,0.999646,0.999882,0.999409,1.000118,541.500000,510.000000,704.000000,778.500000,13.0
75%,62.000000,469.250000,1.000354,1.000590,1.000118,1.000826,846.000000,797.000000,978.000000,1236.000000,13.0
max,72.000000,599.000000,1.002847,1.003094,1.002599,1.003342,2499.000000,2400.000000,2627.000000,2600.000000,13.0
